In [10]:
%pip install bitsandbytes accelerate
%pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Login to Huggingface

In [11]:
from huggingface_hub import notebook_login
notebook_login()

### Load the model

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
import torch

#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", trust_remote_code=True)

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# quantization_config  = BitsAndBytesConfig(
#     load_in_8bit=True, 
#     llm_int8_enable_fp32_cpu_offload=True
# )

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config ,
    device_map="auto",
    trust_remote_code=True
)


model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# next(model.parameters()).dtype

# for name, module in model.named_modules():
#     print(name)

### Configure PEFT

In [13]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 13,631,488 || all params: 7,261,655,040 || trainable%: 0.1877


In [14]:
len(model.peft_config)

1

### Load Dataset

In [15]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoConfig

# load the dataset and split it into train, validation and test sets
dataset = load_dataset("json", data_files="../data/combinations.jsonl", split='train')
shuffled_dataset = dataset.shuffle(seed=42)
train_temp_split = shuffled_dataset.train_test_split(test_size=0.3) #30% for validation and test
temp_dataset = train_temp_split['test']
validation_test_split = temp_dataset.train_test_split(test_size=1/3)# 10% for validation and 20% for test
split_datasets = DatasetDict({
    'train': train_temp_split['train'],
    'validation': validation_test_split['train'],
    'test': validation_test_split['test']
})


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
config = AutoConfig.from_pretrained(model_name)
MAX_LENGTH = 1500


def tokenize_with_loss_mask(example):
    chat_str = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    tokenized = tokenizer(chat_str, truncation=True, padding="max_length", max_length=MAX_LENGTH)

    input_ids = tokenized["input_ids"]
    labels = []

    seq_start_idx = 0 # start of sequence
    total_length = len(chat_str)

    while seq_start_idx < total_length:

        # print("-----------------------")
        seq_end_idx = chat_str.find("</s>", seq_start_idx)+ len("</s>") # end of sequence
        if seq_end_idx == -1:
            break

        sequence = chat_str[seq_start_idx:seq_end_idx]
        end_user_idx = sequence.find("[/INST]")+ len("[/INST]") # end of user message
        end_assistant_idx = sequence.find("</s>")
        user_content = sequence[:end_user_idx]
        assistant_content = sequence[end_user_idx:end_assistant_idx]

        seq_tokens = tokenizer(sequence, add_special_tokens=False)["input_ids"]
        user_tokens = tokenizer(user_content, add_special_tokens=False)["input_ids"]
        assistant_tokens = tokenizer(assistant_content, add_special_tokens=False)["input_ids"]
        
        labels.extend([-100] * len(user_tokens))
        labels.extend(assistant_tokens)
        labels.extend([-100] * (len(seq_tokens) - len(user_tokens) - len(assistant_tokens)))
        
        seq_start_idx = seq_end_idx

    labels = [-100] * (len(input_ids) - len(labels)) + labels  # Pad to max length

    tokenized["labels"] = labels
    if len(tokenized["input_ids"]) > MAX_LENGTH:
        print("Warning: Input sequence exceeds max length for inputs, truncating.")
    if len(tokenized["labels"]) > MAX_LENGTH:
        print("Warning: Input sequence exceeds max length for labels, truncating.")
    if len(tokenized["input_ids"]) != len(tokenized["labels"]):
        print("Error: Input and label lengths do not match after processing.")

    return tokenized

tokenized_dataset = split_datasets.map(tokenize_with_loss_mask, remove_columns=split_datasets["train"].column_names)
print(tokenized_dataset)


Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 70
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})


In [8]:
# from datasets import load_dataset, Dataset
# from transformers import AutoTokenizer


# raw_dataset = load_dataset("json", data_files="../data/combinations.jsonl")["train"]


# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# tokenizer.pad_token = tokenizer.eos_token

# def tokenize(example):
#     model_inputs = tokenizer(
#         tokenizer.apply_chat_template(example["messages"], tokenize=False),
#         truncation=True,
#         padding="max_length",
#         max_length=1024
#     )
#     model_inputs["labels"] = model_inputs["input_ids"].copy()
#     return model_inputs

# tokenized_dataset = raw_dataset.map(tokenize)

# tokenized_dataset

### Tune the model

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=1,#2
    per_device_eval_batch_size=1,#2
    gradient_accumulation_steps=8,#4
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,#10
    eval_strategy="epoch",
    save_strategy="epoch",
    # save_steps=500,
    metric_for_best_model="loss",
    load_best_model_at_end=True,
    output_dir="../output",
    label_names=["labels"],
    disable_tqdm=False,
    gradient_checkpointing=True
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    args=training_args
)

trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,No log,0.841310


KeyboardInterrupt: 

### Evaluate

In [ ]:
import json
from jsonschema import validate, ValidationError

# Check if a string is valid JSON
def is_valid_json(s):
    try:
        json.loads(s)
        return True
    except json.JSONDecodeError:
        return False
    

# Validate a JSON string against a schema
def is_valid_json_schema(json_str, schema):
    try:
        data = json.loads(json_str)
        validate(instance=data, schema=schema)
        return True
    except (json.JSONDecodeError, ValidationError):
        return False
    
# Compare two JSON strings at field level and match field values
def compare_json_field_values(json_ref, json_gen):
    try:
        obj1 = json.loads(json_ref)
        obj2 = json.loads(json_gen)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON input.","percentage": 0}
    matching_fields = []
    matching_values = 0
    total_fields = max(len(obj1), len(obj2))
    for key in obj1:
        if key in obj2:
            matching_fields.append(key)
            if obj1[key] == obj2[key]:
                matching_values += 1
    score = matching_values
    percentage = score / total_fields if total_fields > 0 else 0
    return {
        "matching_fields": matching_fields,
        "matching_values": score,
        "total_fields": total_fields,
        "percentage": percentage
    }



### Save the model